In [ ]:
#install packages and import required packages
!pip install ultralytics -q
import ultralytics
from ultralytics import YOLO
from IPython.display import Image
from datetime import datetime
import os
import glob
import yaml
import csv
import json
import wandb

In [ ]:
#set up
wandb.login(key='WANDB API KEY')
wandb.init(project="Well Identification and Measurement")
HOME=os.getcwd()
!mkdir {HOME}/datasets
%cd {HOME}/datasets
DATASET_LOCATION=f"{HOME}/datasets/Well Identification and Measurement"

In [ ]:
#get dataset
data = {"username":"labanochwo","key":"cb6bdd8fa399054395055df848c757b8"}
with open(f'{HOME}/kaggle.json', 'w') as file:
    json.dump(data, file, indent=4)

!mkdir -p ~/.kaggle
!mv '{HOME}/kaggle.json' ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d labanochwo/well-identification-and-measurement
!unzip -q well-identification-and-measurement.zip -d '{HOME}/datasets/Well Identification and Measurement'
!rm -rf well-identification-and-measurement.zip

In [ ]:
#train model
%cd {HOME}
model = YOLO("yolov5s.pt")
model.train(data=f"{DATASET_LOCATION}/data.yaml", epochs=10, imgsz=512, plots=True)

In [ ]:
#list yolo generated training files
os.listdir(f"{HOME}/runs/detect/train/")

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

In [ ]:
#make predictions
%cd {HOME}
results = model.predict(source=f'{DATASET_LOCATION}/test/images', conf=0.25, save=True, project=f"{HOME}/runs/detect", name="predict", exist_ok=True)

In [ ]:
#display predictions
for image_path in glob.glob(f'{HOME}/runs/detect/predict/*.jpg')[:2]:
    display(Image(filename=image_path, width=1000,height=1000))
    print("\n")

In [ ]:
#create submission file
data = []
for result in results:
    image_name = os.path.basename(result.path)
    class_names =result.names
    for box, cls in zip(result.boxes.xyxy, result.boxes.cls):
        class_name = class_names[int(cls)]
        bbox = box.cpu().tolist()
        record = [image_name, class_name]+bbox
        data.append(record)

with open(f'{HOME}/submission{int(datetime.now().timestamp())}.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    header = ['filename', 'label', 'xmin', 'ymin', 'xmax', 'ymax']
    writer.writerow(header)
    writer.writerows(data)